In [171]:
!pip3 install numpy>=1.20 matplotlib==3.7.5 pandas scikit-learn



/bin/bash: /home/wytcor/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [204]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

# Carregamento do dataset
dataset = pd.read_csv(filepath_or_buffer="/home/wytcor/PROJECTs/mestrado-ufes/lab-life/datasets/zr7vgbcyr2-1/metadata.csv", sep=",")
dataset


,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,...,NaN,NEV,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,5.0,BCC,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,5.0,BCC,True,True,False,False,True,True,PAT_684_1302_588.png,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,PAT_1708,3156,NaN,NaN,NaN,NaN,73,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1708_3156_175.png,False
2294,PAT_46,880,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,12.0,BCC,True,True,False,True,False,False,PAT_46_880_140.png,True
2295,PAT_1343,1217,NaN,NaN,NaN,NaN,74,NaN,NaN,NaN,...,NaN,SEK,False,False,False,False,False,False,PAT_1343_1217_404.png,False
2296,PAT_326,690,False,False,POMERANIA,POMERANIA,58,True,FEMALE,True,...,4.0,BCC,True,False,False,False,False,True,PAT_326_690_823.png,True


Remover os dados que são inválidos

In [205]:
# dataset=dataset.dropna()
dataset=dataset.drop(columns=['lesion_id', 'patient_id', 'img_id'])
dataset.fillna(dataset.mode().iloc[0], inplace=True)
dataset

/tmp/ipykernel_8305/2269315842.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.fillna(dataset.mode().iloc[0], inplace=True)


,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,cancer_history,has_piped_water,...,diameter_1,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,biopsed
0,False,False,POMERANIA,POMERANIA,8,False,FEMALE,False,True,True,...,10.0,5.0,NEV,False,False,False,False,False,False,False
1,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,True,True,...,6.0,5.0,BCC,True,True,False,True,True,True,True
2,False,False,POMERANIA,POMERANIA,77,False,FEMALE,False,True,True,...,10.0,5.0,ACK,True,False,False,False,False,False,False
3,False,False,POMERANIA,POMERANIA,75,False,FEMALE,False,True,True,...,10.0,5.0,ACK,True,False,False,False,False,False,False
4,False,True,POMERANIA,POMERANIA,79,False,MALE,True,False,False,...,5.0,5.0,BCC,True,True,False,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,False,False,POMERANIA,POMERANIA,73,False,FEMALE,False,True,True,...,10.0,5.0,ACK,True,False,False,False,False,False,False
2294,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,True,True,...,13.0,12.0,BCC,True,True,False,True,False,False,True
2295,False,False,POMERANIA,POMERANIA,74,False,FEMALE,False,True,True,...,10.0,5.0,SEK,False,False,False,False,False,False,False
2296,False,False,POMERANIA,POMERANIA,58,True,FEMALE,True,True,False,...,5.0,4.0,BCC,True,False,False,False,False,True,True


Separação entre os dados das features e dos labels

Uso do hot-encoder para converter as variáveis categóricas para inteiras

In [206]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Seleção das features
dataset_features = dataset.drop(columns=['diagnostic'])  # Features

# Identificar variáveis categóricas
categorical_cols = dataset_features.select_dtypes(include=['object']).columns

# Aplicar LabelEncoder para cada coluna categórica
for col in categorical_cols:
    dataset_features[col] = le.fit_transform(dataset_features[col])

# Verificar os primeiros valores após a transformação
print(dataset_features.columns)


Index(['smoke', 'drink', 'background_father', 'background_mother', 'age',
       'pesticide', 'gender', 'skin_cancer_history', 'cancer_history',
       'has_piped_water', 'has_sewage_system', 'fitspatrick', 'region',
       'diameter_1', 'diameter_2', 'itch', 'grew', 'hurt', 'changed', 'bleed',
       'elevation', 'biopsed'],
      dtype='object')


In [191]:
from sklearn.preprocessing import LabelEncoder

# Inicialize o LabelEncoder
le = LabelEncoder()

y = dataset['diagnostic']  # label

# Fit e transform os rótulos em X_train e y_train
y_encoded = le.fit_transform(y)

In [207]:

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(dataset_features, y_encoded, test_size=0.2, shuffle=True)

Uso do modelo de bayers para verificar o impacto da feature no modelo

In [208]:

# Criar o modelo Random Forest
model = RandomForestClassifier(n_estimators=25, random_state=0, max_depth=10)

# Treinar o modelo
model.fit(X_train, y_train)

# Verificar a acurácia no conjunto de teste
accuracy = model.score(X_test, y_test)
print(f'Acurácia: {accuracy * 100:.2f}%')

Acurácia: 77.61%


Análise da importância das features     

In [209]:
import pandas as pd

# Get feature importances
feature_importances = model.feature_importances_

# Map feature importance to feature names
features = X_train.columns
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display top features
print(importance_df.head(10))


              Feature  Importance
21            biopsed    0.189036
4                 age    0.144585
20          elevation    0.089416
16               grew    0.082385
12             region    0.076612
15               itch    0.068762
14         diameter_2    0.055777
13         diameter_1    0.044515
19              bleed    0.037323
2   background_father    0.028886


Usando um modelo de regressão logistica para obter a importância de cada feature

In [210]:
from sklearn.linear_model import LogisticRegression

# Logistic regression coefficients
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Get coefficients
coefficients = model.coef_[0]
importance_df = pd.DataFrame({'Feature': features, 'Coefficient': coefficients})
importance_df = importance_df.sort_values(by='Coefficient', ascending=False)

print(importance_df.head(10))


              Feature  Coefficient
15               itch     1.470648
5           pesticide     0.719713
19              bleed     0.567086
10  has_sewage_system     0.442657
1               drink     0.434920
17               hurt     0.309273
12             region     0.102709
3   background_mother     0.018728
14         diameter_2     0.015433
4                 age     0.010124


/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Avaliação do uso de permutação das features no impacto da acurácia do modelo

In [211]:
from sklearn.inspection import permutation_importance

# Permutation importance
perm_importance = permutation_importance(model, X_test, y_test, scoring='accuracy')

# Map results to features
perm_importance_df = pd.DataFrame({
    'Feature': X_test.columns,
    'Importance': perm_importance.importances_mean
})
perm_importance_df = perm_importance_df.sort_values(by='Importance', ascending=False)

print(perm_importance_df.head(10))


              Feature  Importance
21            biopsed    0.180000
15               itch    0.076957
4                 age    0.066957
20          elevation    0.051304
16               grew    0.043478
19              bleed    0.016522
5           pesticide    0.013043
12             region    0.012174
1               drink    0.009565
3   background_mother    0.009130


Uso do Naive Bayes para verificar o impacto final de cada variável sobre  modelo

Analisando uma feature por vez no treino, e outras sendo adicionadas com o tempo


In [212]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, roc_auc_score
from sklearn.naive_bayes import GaussianNB
# Modelo Naive Bayes
model = GaussianNB()

# Lista para armazenar os resultados
results = []
# Incrementalmente incluir features
for i in range(1, X_train.shape[1] + 1):
    # Selecionar as primeiras `i` features
    X_train_subset = X_train[:, :i] if isinstance(X_train, np.ndarray) else X_train.iloc[:, :i]
    X_test_subset = X_test[:, :i] if isinstance(X_test, np.ndarray) else X_test.iloc[:, :i]
    
    # Treinar o modelo com as features selecionadas
    model.fit(X_train_subset, y_train)
    
    # Obter as probabilidades de saída para o conjunto de teste
    probas = model.predict_proba(X_test_subset)
    # Verificando a estrutura das probabilidades
    # print(probas[:,1])

    # Predições do modelo
    predictions = model.predict(X_test_subset)
    
    # Calcular métricas (roc_auc_score usa as probabilidades da classe positiva)
    roc_auc = roc_auc_score(y_test, probas, multi_class='ovr', average='macro') if len(np.unique(y_test)) > 2 else roc_auc_score(y_test, probas[:, 1])
    
    # Armazenar resultados
    results.append({
        'features_used': list(range(1, i + 1)),  # Índices das features usadas
        'probabilities': probas,
        'predictions': predictions,
        'true_labels': y_test,
        'accuracy_score': accuracy_score(y_test, predictions),
        'balanced_accuracy_score': balanced_accuracy_score(y_test, predictions),
        'f1_score': f1_score(y_test, predictions, average='weighted'), 
        'precision_score': precision_score(y_test, predictions, average='weighted'),
        'roc_auc_score': roc_auc
    })


/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

Exibir os dados da filtragem 

In [214]:

# Exibir os resultados
for result in results:
    print(f"accuracy_score usadas: {result['accuracy_score']}")
    print(f"balanced_accuracy_score:\n{result['balanced_accuracy_score']}")
    print(f"f1_score usadas: {result['f1_score']}")
    print(f"precision_score:\n{result['precision_score']}")
    print(f"roc_auc_score:\n{result['roc_auc_score']}")

    print("-" * 40)


accuracy_score usadas: 0.34347826086956523
balanced_accuracy_score:
0.18555035650623886
f1_score usadas: 0.23807639385669752
precision_score:
0.3192378188977908
roc_auc_score:
0.5541502819179556
----------------------------------------
accuracy_score usadas: 0.39565217391304347
balanced_accuracy_score:
0.20604946524064172
f1_score usadas: 0.3121304852800081
precision_score:
0.3249032355041149
roc_auc_score:
0.5939483078720621
----------------------------------------
accuracy_score usadas: 0.3391304347826087
balanced_accuracy_score:
0.2467214158390629
f1_score usadas: 0.29867426727254337
precision_score:
0.34256952261034906
roc_auc_score:
0.6717066762095122
----------------------------------------
accuracy_score usadas: 0.33260869565217394
balanced_accuracy_score:
0.25484625668449196
f1_score usadas: 0.2955327480912713
precision_score:
0.34213132894494785
roc_auc_score:
0.6912003814416017
----------------------------------------
accuracy_score usadas: 0.41956521739130437
balanced_accura

Exportar os dados em um formato de arquivo csv

In [219]:
import pandas as pd

def save_results_to_csv(results, filename='model_results.csv'):
    # Criação de uma lista para armazenar os dados formatados para o CSV
    rows = []
    
    # Itera sobre os resultados e organiza as métricas e as features usadas
    for result in results:
        row = {
            'features_used': len(result['features_used']),
            'accuracy_score': result['accuracy_score'],
            'balanced_accuracy_score': result['balanced_accuracy_score'],
            'f1_score': result['f1_score'],
            'precision_score': result['precision_score'],
            'roc_auc_score': result['roc_auc_score']
        }
        rows.append(row)
    
    # Convertendo a lista de resultados para um DataFrame
    df = pd.DataFrame(rows)
    
    # Salvando o DataFrame em um arquivo CSV
    df.to_csv(filename, index=False)

In [221]:
save_results_to_csv(results, './results/naive-bayes/model_results_mode_as_nan_replacer.csv')